In [1]:
import numpy as np

# Define prior probabilities of getting pregnant for each method
prior_probs = np.array([0.15, 0.2, 0.4])

# Define likelihood ratios for each method
natural_lr = 0.1
iui_lr = 0.3
ivf_lr = 0.7

# Define test results (1 for positive, 0 for negative)
natural_test = 1
iui_test = 0
ivf_test = 1

# Calculate posterior probabilities using Bayes' theorem
natural_post = (natural_lr * prior_probs[0]) / ((natural_lr * prior_probs[0]) + ((1 - prior_probs[0]) * (1 - natural_lr)))
iui_post = (iui_lr * prior_probs[1]) / ((iui_lr * prior_probs[1]) + ((1 - prior_probs[1]) * (1 - iui_lr)))
ivf_post = (ivf_lr * prior_probs[2]) / ((ivf_lr * prior_probs[2]) + ((1 - prior_probs[2]) * (1 - ivf_lr)))

# Calculate overall probability of pregnancy
pregnancy_prob = np.sum([natural_post, iui_post, ivf_post])

# Print results
print("Posterior probabilities: ", natural_post, iui_post, ivf_post)
print("Overall probability of pregnancy: ", pregnancy_prob)

Posterior probabilities:  0.01923076923076923 0.09677419354838711 0.6086956521739131
Overall probability of pregnancy:  0.7247006149530695


In [2]:
# Incorporate egg count numbers and past ovulation tracking
egg_count = 10 # number of eggs
ovulation_tracking = 0.8 # probability of ovulating
natural_lr *= (ovulation_tracking * egg_count) / 100
iui_lr *= (ovulation_tracking * egg_count) / 100
ivf_lr *= (ovulation_tracking * egg_count) / 100

# Print results
print("Posterior probabilities: ", natural_post, iui_post, ivf_post)
print("Overall probability of pregnancy: ", pregnancy_prob)

Posterior probabilities:  0.01923076923076923 0.09677419354838711 0.6086956521739131
Overall probability of pregnancy:  0.7247006149530695


In [3]:
# Incorporate estimated time to pregnancy
natural_time = 6 # months to conceive naturally on average
iui_time = 9 # months to conceive with IUI on average
ivf_time = 12 # months to conceive with IVF on average
natural_lr /= natural_time
iui_lr /= iui_time
ivf_lr /= ivf_time

# Calculate estimated time to pregnancy
natural_time_to_pregnancy = 1 / natural_post * natural_time
iui_time_to_pregnancy = 1 / iui_post * iui_time
ivf_time_to_pregnancy = 1 / ivf_post * ivf_time

# Print results
print("Posterior probabilities: ", natural_post, iui_post, ivf_post)
print("Overall probability of pregnancy: ", pregnancy_prob)
print("Estimated time to pregnancy (months):", natural_time_to_pregnancy, iui_time_to_pregnancy, ivf_time_to_pregnancy)

Posterior probabilities:  0.01923076923076923 0.09677419354838711 0.6086956521739131
Overall probability of pregnancy:  0.7247006149530695
Estimated time to pregnancy (months): 312.00000000000006 92.99999999999999 19.714285714285715


In [4]:
from flask import Flask, request, render_template
import numpy as np

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/predict', methods=['POST'])
def predict():
    # Get inputs from form data
    egg_count = int(request.form['egg_count'])
    ovulation_tracking = float(request.form['ovulation_tracking'])
    natural_test = int(request.form['natural_test'])
    iui_test = int(request.form['iui_test'])
    ivf_test = int(request.form['ivf_test'])
    
    # Define prior probabilities of getting pregnant for each method
    prior_probs = np.array([0.15, 0.2, 0.4])

    # Define likelihood ratios for each method
    natural_lr = 0.1
    iui_lr = 0.3
    ivf_lr = 0.7

    # Incorporate egg count numbers and past ovulation tracking
    natural_lr *= (ovulation_tracking * egg_count) / 100
    iui_lr *= (ovulation_tracking * egg_count) / 100
    ivf_lr *= (ovulation_tracking * egg_count) / 100

    # Incorporate estimated time to pregnancy
    natural_time = 6 # months to conceive naturally on average
    iui_time = 9 # months to conceive with IUI on average
    ivf_time = 12 # months to conceive with IVF on average
    natural_lr /= natural_time
    iui_lr /= iui_time
    ivf_lr /= ivf_time

    # Apply test results
    if natural_test == 1:
        natural_lr *= 10
    else:
        natural_lr *= 0.1
    if iui_test == 1:
        iui_lr *= 3
    else:
        iui_lr *= 0.3
    if ivf_test == 1:
        ivf_lr *= 7
    else:
        ivf_lr *= 0.7

    # Calculate posterior probabilities using Bayes' theorem
    natural_post = (natural_lr * prior_probs[0]) / ((natural_lr * prior_probs[0]) + ((1 - prior_probs[0]) * (1 - natural_lr)))
    iui_post = (iui_lr * prior_probs[1]) / ((iui_lr * prior_probs[1]) + ((1 - prior_probs[1]) * (1 - iui_lr)))
    ivf_post = (ivf_lr * prior_probs[2]) / ((ivf_lr * prior_probs[2]) + ((1 - prior_probs[2]) * (1 - ivf_lr)))

    # Calculate overall probability of pregnancy
    pregnancy_prob = np.sum([natural_post, iui_post, ivf_post])

    # Calculate estimated time to pregnancy
    natural_time_to_pregnancy = 1 / natural_post * natural_time
    iui_time_to_pregnancy = 1 / iui_post * iui_time
    ivf_time_to_pregnancy = 1 / ivf_post * ivf_time

In [5]:
    # Prepare output for display
    output = f"Your chances of getting pregnant with natural means are {natural_post:.2%}. The estimated time to pregnancy is {natural_time_to_pregnancy:.1f} months.\n"
    output += f"Your chances of getting pregnant with IUI are {iui_post:.2%}. The estimated time to pregnancy is {iui_time_to_pregnancy:.1f} months.\n"
    output += f"Your chances of getting pregnant with IVF are {ivf_post:.2%}. The estimated time to pregnancy is {ivf_time_to_pregnancy:.1f} months.\n"
    output += f"Overall, your chance of getting pregnant through any method is {pregnancy_prob:.2%}."


<form method="post" action="/predict">
    <label for="age">Age:</label>
    <input type="number" id="age" name="age" required>
    <br>
    <label for="fsh">FSH level:</label>
    <input type="number" id="fsh" name="fsh" required>
    <br>
    <label for="amh">AMH level:</label>
    <input type="number" id="amh" name="amh" required>
    <br>
    <label for="ovulation">Days since last ovulation:</label>
    <input type="number" id="ovulation" name="ovulation" required>
    <br>
    <label for="pregnancies">Number of previous pregnancies:</label>
    <input type="number" id="pregnancies" name="pregnancies" required>
    <br>
    <label for="miscarriages">Number of previous miscarriages:</label>
    <input type="number" id="miscarriages" name="miscarriages" required>
    <br>
    <label for="attempts">Number of previous attempts:</label>
    <input type="number" id="attempts" name="attempts" required>
    <br>
    <label for="method">Preferred method (natural, iui, or ivf):</label>
    <select id="method" name="method" required>
        <option value="natural">Natural</option>
        <option value="iui">IUI</option>
        <option value="ivf">IVF</option>
    </select>
    <br>
    <input type="submit" value="Submit">
</form>


In [ ]:
from flask import Flask, request, render_template

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    age = int(request.form['age'])
    fsh = float(request.form['fsh'])
    amh = float(request.form['amh'])
    ovulation = int(request.form['ovulation'])
    pregnancies = int(request.form['pregnancies'])
    miscarriages = int(request.form['miscarriages'])
    attempts = int(request.form['attempts'])
    method = request.form['method']

    # Use the code above to compute the output
    # ...

    # Prepare the output for display
    output = # ...

    # Render an HTML template that displays the output to the user
    return render_template('output.html', output=output)


<!DOCTYPE html>
<html>
<head>
    <title>Output</title>
</head>
<body>
    <h1>Pregnancy prediction results:</h1>
    <p>Based on the information you provided, your chances of getting pregnant are:</p>
    <ul>
        <li>Naturally: {{ natural_chance }}%</li>
        <li>IUI: {{ iui_chance }}%</li>
        <li>IVF: {{ ivf_chance }}%</li>
    </ul>
    <p>It is estimated that it will take:</p>
    <ul>
        <li>Naturally: {{ natural_time }} months</li>
        <li>IUI: {{ iui_time }} months</li>
        <li>IVF: {{ ivf_time }} months</li>
    </ul>
</body>
</html>
